# Deriving Quaternion Rotation Sequences

To track the rotation sequence betweem a rotating body and a fixed reference frame you can use either Euler angles in a rotation matrix or you can use Quaternions. Quaternerions are generally preferred from a mathematical standpoint, but less intuitive to most people.

## R321

$$
r^b = R_{321} \cdot r^I = R_1(x) R_2(y) R_3(z) \cdot r^I
$$

$$
R321 =
\left[\begin{matrix}\cos{\left(y \right)} \cos{\left(z \right)} & \sin{\left(z \right)} \cos{\left(y \right)} & - \sin{\left(y \right)}\\
\sin{\left(x \right)} \sin{\left(y \right)} \cos{\left(z \right)} - \sin{\left(z \right)} \cos{\left(x \right)} & \sin{\left(x \right)} \sin{\left(y \right)} \sin{\left(z \right)} + \cos{\left(x \right)} \cos{\left(z \right)} & \sin{\left(x \right)} \cos{\left(y \right)}\\
\sin{\left(x \right)} \sin{\left(z \right)} + \sin{\left(y \right)} \cos{\left(x \right)} \cos{\left(z \right)} & - \sin{\left(x \right)} \cos{\left(z \right)} + \sin{\left(y \right)} \sin{\left(z \right)} \cos{\left(x \right)} & \cos{\left(x \right)} \cos{\left(y \right)}\end{matrix}\right]
$$

$$
\begin{align}
yaw &= z = \arctan(\frac {R_{12}}{R_{11}}) \\
pitch &= y = -\sin(R_{13}) \\
roll &= x = \arctan(\frac {R_{23}} {R_{33}})
\end{align}
$$

## Quaternions

Individual rotations for roll, pitch and yaw:

$$
\begin{align}
q_x &= \begin{bmatrix} \cos \frac {roll}{2} & \sin \frac {roll}{2} & 0 & 0 \end{bmatrix} \\
q_y &= \begin{bmatrix} \cos \frac {pitch}{2} & 0 & \sin \frac {pitch}{2} & 0 \end{bmatrix} \\
q_z &= \begin{bmatrix} \cos \frac {yaw}{2} & 0 & 0 & \sin \frac {yaw}{2} \end{bmatrix}
\end{align}
$$

To compose the Aerospace sequence of roll, pitch, then yaw:

$$
q = q_z q_y q_x
$$

$$
\begin{array}{rcl}
q_w &= \cos\frac{yaw}{2} \cos\frac{pitch}{2} \cos\frac{roll}{2} + \sin\frac{yaw}{2} \sin\frac{pitch}{2} \sin\frac{roll}{2} \\ && \\
q_x &= \cos\frac{yaw}{2} \cos\frac{pitch}{2} \sin\frac{roll}{2} - \sin\frac{yaw}{2} \sin\frac{pitch}{2} \cos\frac{roll}{2} \\ && \\
q_y &= \cos\frac{yaw}{2} \sin\frac{pitch}{2} \cos\frac{roll}{2} + \sin\frac{yaw}{2} \cos\frac{pitch}{2} \sin\frac{roll}{2} \\ && \\
q_z &= \sin\frac{yaw}{2} \cos\frac{pitch}{2} \cos\frac{roll}{2} - \cos\frac{yaw}{2} \sin\frac{pitch}{2} \sin\frac{roll}{2}
\end{array}
$$

## References

- [Euler Angles](https://academicflight.com/articles/kinematics/rotation-formalisms/euler-angles/)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy
from sympy import symbols, simplify
from sympy import latex
from sympy import Matrix
from sympy import cos, sin, atan2

import numpy as np
from numpy.testing import assert_allclose
np.set_printoptions(precision=1)
np.set_printoptions(suppress=True)
from pyquat import Quaternion as pyquat

In [3]:
# quaternion: w,x,y,z
# OR
# x: roll
# y: pitch
# z: yaw
w,x,y,z = symbols("w x y z")

In [4]:
# rotate about z-axis
Rz = np.array([
    [cos(z), sin(z), 0],
    [-sin(z), cos(z),0],
    [0,0,1]
])

# rotate about y-axis
Ry = np.array([
    [cos(y), 0, -sin(y)],
    [0,1,0],
    [sin(y),0,cos(y)]
])

# rotate about the x-axis
Rx = np.array([
    [1,0,0],
    [0,cos(x),sin(x)],
    [0, -sin(x),cos(x)]
])

In [5]:
# Standard Aerospace rotation for
# inertial to body transform
R321 = Matrix(Rx @ Ry @ Rz)
R321

Matrix([
[                       cos(y)*cos(z),                         sin(z)*cos(y),       -sin(y)],
[sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(y)*sin(z) + cos(x)*cos(z), sin(x)*cos(y)],
[sin(x)*sin(z) + sin(y)*cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x), cos(x)*cos(y)]])

In [6]:
# body to inertial is the transpose
R321.T

Matrix([
[cos(y)*cos(z), sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(z) + sin(y)*cos(x)*cos(z)],
[sin(z)*cos(y), sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[      -sin(y),                        sin(x)*cos(y),                         cos(x)*cos(y)]])

In [7]:
# this can also be done by taking the transpose of the rearranged 
# rotation matricies ... interesting
Matrix(Rz.T @ Ry.T @ Rx.T)

Matrix([
[cos(y)*cos(z), sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(z) + sin(y)*cos(x)*cos(z)],
[sin(z)*cos(y), sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[      -sin(y),                        sin(x)*cos(y),                         cos(x)*cos(y)]])

## Quaternions

This is a Hamilton quaternion and is composed of a real part (w) and an imaginary part (x,y,z). Just like typical complex numbers in mathematics.

In [8]:
qx = pyquat(cos(x/2),sin(x/2),       0,      0)
qy = pyquat(cos(y/2),       0,sin(y/2),      0)
qz = pyquat(cos(z/2),       0,       0,sin(z/2))

In [10]:
q321=qz*qy*qx

In [11]:
print("q.w: ",q321.w)
print("q.x: ",q321.x)
print("q.y: ",q321.y)
print("q.z: ",q321.z)

q.w:  sin(x/2)*sin(y/2)*sin(z/2) + cos(x/2)*cos(y/2)*cos(z/2)
q.x:  sin(x/2)*cos(y/2)*cos(z/2) - sin(y/2)*sin(z/2)*cos(x/2)
q.y:  sin(x/2)*sin(z/2)*cos(y/2) + sin(y/2)*cos(x/2)*cos(z/2)
q.z:  -sin(x/2)*sin(y/2)*cos(z/2) + sin(z/2)*cos(x/2)*cos(y/2)


In [12]:
euler = np.array(q321.to_euler())
print(simplify(euler[0]))
print(simplify(euler[1]))
print(simplify(euler[2]))

atan2(0.5*sin(x - y) + 0.5*sin(x + y), 0.5*cos(x - y) + 0.5*cos(x + y))
asin(1.0*sin(y))
atan2(-0.5*sin(y - z) + 0.5*sin(y + z), 0.5*cos(y - z) + 0.5*cos(y + z))


## Compare R321 to q321

To compare we will look at the rotation matrix (DCM) that they produce. This is sort the of the "universal key" between the two forms.

In [13]:
rot = np.array(q321.to_rot())
rot = simplify(rot)
rot

[[cos(y - z)/2 + cos(y + z)/2, sin(x)*sin(y)*cos(z) - sin(z)*cos(x), sin(x)*sin(z) + sin(y)*cos(x)*cos(z)], [-sin(y - z)/2 + sin(y + z)/2, sin(-x + y + z)/4 + sin(x - y + z)/4 + sin(x + y - z)/4 - sin(x + y + z)/4 + cos(x - z)/2 + cos(x + z)/2, -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)], [-sin(y), sin(x - y)/2 + sin(x + y)/2, cos(x - y)/2 + cos(x + y)/2]]

In [24]:
# this is from rotating body to fixed frame
qR = rot.subs(x,np.pi/3).subs(y,-np.pi/4).subs(z,np.pi/5)
qR

[[0.572061402817684, -0.789312333510914, 0.223006259046285], [0.415626937777453, 0.044565010575065, -0.908442738110763], [0.707106781186547, 0.612372435695794, 0.353553390593274]]

In [25]:
# this is from fixed frame to rotating body
rR = R321.subs(x,np.pi/3).subs(y,-np.pi/4).subs(z,np.pi/5)
rR

Matrix([
[ 0.572061402817684,  0.415626937777453, 0.707106781186547],
[-0.789312333510914,  0.044565010575065, 0.612372435695795],
[ 0.223006259046285, -0.908442738110763, 0.353553390593274]])

In [26]:
# the transpose reverses the directions, so now
# this is from rotating body to fixed frame. Now
# this matches the q321.to_rot() answer (qR)
# above ... yay!
rR.T

Matrix([
[0.572061402817684, -0.789312333510914,  0.223006259046285],
[0.415626937777453,  0.044565010575065, -0.908442738110763],
[0.707106781186547,  0.612372435695795,  0.353553390593274]])